# Stundatöfluröðun

### **Fyrsta líkan**

$$
x_{n,s}\in\{0,1\},\quad n\in N_\text{von},\quad s\in\{0,\dots, 7\}
$$
þar sem $n$ er námskeið og $s$ slott.

Sérhver tími þarf að vera í einu slotti
$$
\sum_{s\in S}x_{n,s}=1,\quad \forall n\in N_\text{von}
$$
Engin skyldunámskeið mega vera kennd í sama stokk
$$
\sum_{n\in \text{svið, braut, ár}}T_nx_{n,s} \leq 5,\quad \forall (s\ne 0 \land s\in S) \land r\in R
$$
Bætum við skorðu þannig ef tvo skyldunámskeið hafa saman minna en 5 tíma má setja þau í sama stokk.


**Hlutir sem þarf að skoða í flóknari líkani**
- Fjöldi nemenda < stofur fjöldi
- Ef bara tveir tímar (3) í hvort slottið á að fara
- $N_n$ fjöldi nema
- $\overline N_r$ stærð stofu
- þarf ekki að hafa kennara árekstra
- breyta $x_{n,s,\dots,r}$ $n$ námslota, $s$ stokkur og $r$ stofa
- takmarka fjarlægð milli kennslustunda
- takmarka tíma milli kennslustunda
- lágmarka árekstri
- fyrir áresktra $\min \sum_{n_1< n_2} C_{n_1,n_2}*z_{n_1m,n_2}$ þar sem $z$ segir til um hvort námskeið séu kennd á sama tíma

In [2]:
from student_hitt.load_data import load
import os
import pandas as pd
os.chdir("student_hitt")
room_df, fid_df, clash_df, name_df, schedule_df = load()

In [3]:
# Apply initial filters, VON,
filtered_df = fid_df[
    (fid_df['scid'] == 50) &
    (fid_df['category'] == 'M') &
    (fid_df['lcid'].isin(schedule_df['lcid'])) &
    (fid_df['year'] != -1)
]

# Define a filtering function
def fid_df_to_fixed_dict(df):
    '''Transforms fid_df to dict from (pid, fid, year) to set of mandatory lcids'''
    CORE_ID = -1
    gdf = df[df['category'].eq('M')].groupby(['pid', 'fid', 'year'])['lcid'].unique().apply(set)
    data: dict[tuple[str, int, int], set[str]] = gdf.to_dict()
    pidfid_to_lcid = tuple(data.items())

    # Cut out lcids from core:
    for (pid, fid, year), lcids in pidfid_to_lcid:
        if fid == CORE_ID:
            continue
        if (pid, CORE_ID, year) in data:
            data[pid, CORE_ID, year] =  data[pid, CORE_ID, year] - lcids

    # Add what is left in core to all other fields with (pid, year):
    for (pid, fid, year), lcids in pidfid_to_lcid:
        if fid == CORE_ID:
            continue
        if (pid, CORE_ID, year) in data:
            data[pid, fid, year] = data[pid, CORE_ID, year].union(lcids)

    return data


# Apply the filtering logic
grouped = fid_df_to_fixed_dict(filtered_df)

# Convert to dictionary
result_dict = {key: list(value) for key, value in grouped.items()}

# Convert to datetime (if not already)
schedule_df['start'] = pd.to_datetime(schedule_df['start'], format='%H:%M:%S')
schedule_df['end'] = pd.to_datetime(schedule_df['end'], format='%H:%M:%S')

# Calculate duration in minutes
schedule_df['duration_minutes'] = (schedule_df['end'] - schedule_df['start']).dt.total_seconds() / 60

# Group by lcid and sum the duration
# Sum and convert to DataFrame
total_minutes_per_lcid = (
    schedule_df.groupby('lcid')['duration_minutes'].sum().reset_index(name='duration_minutes')
)

# Now you can compute periods
total_minutes_per_lcid['periods'] = (total_minutes_per_lcid['duration_minutes'] / 46).round().astype(int)
total_minutes_per_lcid['periods'] = total_minutes_per_lcid['periods'].clip(upper=5)
T = dict(zip(total_minutes_per_lcid['lcid'], total_minutes_per_lcid['periods']))

# Get the student count
sidcount_df = schedule_df[['lcid', 'sidcount']].drop_duplicates(subset='lcid')
sidcount_df['sidcount'] = sidcount_df['sidcount'].fillna(0).astype(int)

# Convert to dictionary
Ns = dict(zip(sidcount_df['lcid'], sidcount_df['sidcount']))

buildings = ['Aðalbygging', 'Askja',
       'Árnagarður',  'gimli', 'Gimli', 'Gróska',
       'Háskólabíó', 'Háskólatorg', 'Lögberg', 'Oddi', 'Saga', 'Tæknigarður',
       'Veröld - Hús Vigdísar', 'VR-1', 'VR-2', 'VR-3']

filtered_room_df = room_df[
    (room_df["building_name"].isin(buildings)) &
    (room_df["type"] == 3)
]

room_dict = (
    filtered_room_df.dropna(subset=["capacity"])
               .set_index("room_name")["capacity"]
               .to_dict()
)

In [11]:
room_dict['A-050']

30.0

In [5]:
# The mandatory courses in the different study programs
for a in result_dict.keys():
  print(a,result_dict[a])

('BYG441', 105, 1) ['5056UMV205M20250']
('EFN221', -1, 1) ['5055EFN208G20250']
('EFN231', -1, 1) ['5055EFN210G20250', '5055STÆ203G20250', '5055EFN202G20250', '5055STÆ205G20250', '5055STÆ209G20250', '5055EÐL208G20250', '5055EFN208G20250']
('EFN231', -1, 2) ['5055EFN414G20250', '5055EFN407G20250', '5055EFN404G20250', '5055EFN209G20250', '5055STÆ211G20250', '5055EFN413G20250', '5055EFN406G20250', '5055EFN410G20250']
('EFN231', -1, 3) ['5055EFN612M20250']
('EFN241', -1, 1) ['5055EFN210G20250', '5055EFN211G20250', '5055EFN209G20250', '5055EFN202G20250', '5055STÆ205G20250', '5055STÆ209G20250', '5055EÐL208G20250', '5055EFN208G20250']
('EFN241', -1, 2) ['5055EFN414G20250', '5055EFN407G20250', '5055EFN412G20250', '5055EFN404G20250', '5055EFN413G20250', '5055EFN406G20250', '5055EFN410G20250']
('EFN241', -1, 3) ['5055EFN612M20250']
('EVF264', -1, 1) ['5055EVF401G20250', '5055EFN209G20250', '5055EFN202G20250', '5055STÆ205G20250', '5051EVF201M20250', '5055EÐL208G20250']
('EVF264', -1, 2) ['5051EVF4

In [6]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB

In [16]:
# Mengi
S = list(range(8)) # stokkur 0 er utan stokkar
N = list(Ns.keys()) # námskeið N == T.keys(), fjöldi tíma er í T
R = list(room_dict.keys()) # kennslustofur

S0 = list(range(1,8))

In [8]:
model = gp.Model()
x = model.addVars(N,S,R,vtype=GRB.BINARY)

# sérhvert námskeið aðeins kennt í einni stofu og í einu slotti
model.addConstrs(
    (gp.quicksum(x[n, s, r] for s in S for r in R) == 1 for n in N),
    name="course_assignment"
)

# engin skyldunámskeið kennd á sama tíma
for namsbraut in result_dict:
    for s in S:
        if (s!=0):
            model.addConstr(
                gp.quicksum(x[n, s, r] * T[n] for n in result_dict[namsbraut] for r in R) <= 5,
                name=f"max_classes_{namsbraut}_slot{s}"
            )

# námskeið í sama slotti geta ekki verið í sömu stofu.
for s in S:
    if (s!=0):
        for r in R:
            model.addConstr(
                gp.quicksum(x[n, s, r] for n in N) <= 1,
                name=f"one_course_per_room_slot_{s}_{r}"
            )

# námskeið þarf að vera kennt í stofu sem það passar í

model.addConstrs(
    (x[n, s, r] * Ns[n] <= room_dict[r] for n in N for s in S for r in R),
    name="room_capacity"
)

model.setObjective(
    gp.quicksum(x[n, 0, r] * T[n] for n in N for r in R),
    GRB.MINIMIZE
)

model.optimize()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2638823


Academic license 2638823 - for non-commercial use only - registered to jr___@hi.is
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[x86] - Darwin 23.6.0 23H420)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Academic license 2638823 - for non-commercial use only - registered to jr___@hi.is
Optimize a model with 2403 rows, 747856 columns and 1627980 nonzeros
Model fingerprint: 0xaffe447b
Variable types: 0 continuous, 747856 integer (747856 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 4426.0000000
Presolve removed 301 rows and 92395 columns
Presolve time: 4.47s
Presolved: 2102 rows, 655461 columns, 1469967 nonzeros
Variable types: 0 continuous, 655461 integer (655461 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dua

In [9]:
# prenta út niðurstöður
for n in ['5055EFN406G20250', '5055EFN404G20250', '5055EFN414G20250', '5055EFN209G20250', '5055EFN413G20250', '5055EFN410G20250', '5055EFN407G20250', '5055STÆ211G20250']:# námsbraut sem éh er að skoða
    for s in S:
        for r in R:
            if x[n,s,r].X>0.5:
                print(f'{n} er í stokki {s} og stofu {r}')
                print(T[n])
    
    

5055EFN406G20250 er í stokki 4 og stofu VHV-228
5
5055EFN404G20250 er í stokki 3 og stofu V02-262, heimast.
5
5055EFN414G20250 er í stokki 6 og stofu O-103
5
5055EFN209G20250 er í stokki 1 og stofu VHV-224
5
5055EFN413G20250 er í stokki 5 og stofu S-261
5
5055EFN410G20250 er í stokki 2 og stofu V02-158
5
5055EFN407G20250 er í stokki 7 og stofu O-103
5
5055STÆ211G20250 er í stokki 0 og stofu V02-155
2


In [10]:
for r in R:
    print("stofa "+r)
    for n in N:
        for s in S:
            if x[n,s,r].X>0.5:
                print(f"Námskeið {n} í slotti {s}")

stofa A-050
Námskeið 5054RAF620M20250 í slotti 3
Námskeið 5054TUG201G20250 í slotti 5


Námskeið 1011FÉL089F20250 í slotti 1
Námskeið 2023LEI602G20250 í slotti 4
Námskeið 2023SJÚ611G20250 í slotti 7
Námskeið 2021HJÚ225G20250 í slotti 2
Námskeið 5055EFN215G20250 í slotti 6
stofa A-051
Námskeið 5054RAF612G20250 í slotti 2
Námskeið 4042KME006F20250 í slotti 7
Námskeið 3031ÍTA250G20250 í slotti 1
Námskeið 1016VIÐ293F20250 í slotti 5
Námskeið 1012FRG210F20250 í slotti 4
Námskeið 2021HJÚ212G20250 í slotti 3
Námskeið 1012FRG238F20250 í slotti 6
stofa A-052
Námskeið 5051TÖL605M20250 í slotti 4
Námskeið 5052JAR414M20250 í slotti 1
Námskeið 1011FFR102M20250 í slotti 5
Námskeið 1014LÖG219F20250 í slotti 3
Námskeið 1011FÉL262G20250 í slotti 2
Námskeið 2026TSM204G20250 í slotti 6
Námskeið 2026TAN604M20250 í slotti 7
stofa A-220
Námskeið 5055EÐL401G20250 í slotti 7
Námskeið 5055MAS202M20250 í slotti 1
Námskeið 5054MEK606G20250 í slotti 4
Námskeið 1016VIÐ278F20250 í slotti 2
Námskeið 1016VIÐ289F20250 í slotti 5
Námskeið 2023LÆK615F20250 í slotti 6
Námskeið 2026TAN516G20250 í slotti 3
st